In [1]:
import pennylane as qml
import numpy as np
from scipy.optimize import minimize

In [2]:
def codificar_amistades(amistades):
    # Transforma el array de nombres de personas a array de índices
    return np.unique(amistades, return_inverse=True)

In [3]:
def construir_circuito(personas_idxs, amistades_idxs):
    dev = qml.device("default.qubit", wires=personas_idxs)

    # Codifica Hamiltoniano de https://arxiv.org/pdf/1904.00051 (pág 2) convertido a Ising.
    # Se podría usar https://pennylane.ai/qml/demos/tutorial_qaoa_intro para simplificar quizás?
    def U_C(gamma):

        # Términos lineales
        _, cantidad_amistades = np.unique(amistades_idxs.flatten(), return_counts=True)
        for persona_idx in personas_idxs:
            cantidad = cantidad_amistades[persona_idx]
            qml.RZ(-2 * (0.25 * cantidad - 1) * gamma, wires = persona_idx)

        # Términos cuadráticos
        for (persona_idx_1, persona_idx_2) in amistades_idxs:
            qml.CNOT(wires = [persona_idx_1, persona_idx_2])
            qml.RZ(-2 * (0.25) * gamma, wires = persona_idx_2)
            qml.CNOT(wires = [persona_idx_1, persona_idx_2])

    def U_B(beta):
        for persona_idx in personas_idxs:
            qml.RX(-2 * beta, wires = persona_idx)

    @qml.qnode(dev)
    def circuit(gamma, beta):
        for persona_idx in personas_idxs:
            qml.Hadamard(wires = persona_idx)

        for _ in range(3):
            U_C(gamma)
            U_B(beta)
        
        return qml.probs(wires = personas_idxs)

    return circuit

In [4]:
def optimizar_parametros(circuito):
    parametros_iniciales = [0.15, 0.15]
    # La optimización es de minimización, entonces se arma una función que toma
    # el negativo de la probabilidad de obtener el resultado de máxima energía.
    optimizable = lambda params : -np.max(circuito(params[0], params[1]))
    parametros_optimizados = minimize(optimizable, parametros_iniciales).x
    return parametros_optimizados[0], parametros_optimizados[1]

In [5]:
def a_quien_contratar(amistades):
    # Codificar amistades
    personas, amistades_idxs = codificar_amistades(amistades)
    personas_idxs = range(personas.size)

    # Generar circuito y parámetros
    circuito = construir_circuito(personas_idxs, amistades_idxs)
    gamma, beta = optimizar_parametros(circuito)

    # Obtener resultado
    probs = circuito(gamma, beta)
    resultado = np.argmax(probs)

    # Decodificar resultado y devolver personas elegidas
    personas_elegidas = []
    for idx, valor_binario in enumerate(bin(resultado)[2:]):
        if valor_binario == "0":
            personas_elegidas.append(str(personas[idx]))
    return personas_elegidas

## Prueba

In [6]:
amistades = [["Mario", "Sarah"],
            ["Mario", "Raul"],
            ["Mario", "Ana"],
            ["Enrique", "Sarah"],
            ["Enrique", "Raul"],
            ["Saul", "Ana"]]

a_quien_contratar(amistades)

[]

In [7]:
amistades = [["Mario", "Sarah"]]

a_quien_contratar(amistades)

[]